### Chat bot

In [1]:
import os
from dotenv import load_dotenv

from langchain import PromptTemplate, LLMChain

load_dotenv()

key=os.environ['OPENAI_API_KEY']

### Creando un bot simple

In [8]:
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

from langchain.chains import LLMChain, ConversationChain
from langchain.chat_models import ChatOpenAI


In [10]:
chat = ChatOpenAI(openai_api_key=key)
messages = [
    SystemMessage(content= "Tu eres un asistente que ayuda al usuario a planear y optimizar su itinerario"),
    HumanMessage(content= "Necesito planear un viaje por 5 días por bariloche, argentina")
]

In [11]:
output = chat(messages=messages)
print(output)

/tmp/ipykernel_420324/2042678014.py:1: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  output = chat(messages=messages)


content='¡Claro! Para planear un viaje de 5 días por Bariloche, Argentina, te recomendaría considerar las siguientes actividades y lugares para visitar:\n\nDía 1:\n- Llegada a Bariloche\n- Paseo por el centro de la ciudad\n- Visita a la Catedral de San Carlos de Bariloche\n- Disfrutar de una cena en un restaurante local\n\nDía 2:\n- Excursión al Cerro Catedral para disfrutar de la vista panorámica de la ciudad y realizar actividades como trekking o esquí, dependiendo de la temporada\n- Visita al Lago Gutiérrez para relajarse y disfrutar de la naturaleza\n\nDía 3:\n- Recorrido por la Ruta de los Siete Lagos, visitando lagos como el Nahuel Huapi, Correntoso y Espejo, entre otros\n- Parada en Villa La Angostura para conocer este pintoresco pueblo\n\nDía 4:\n- Visita al Parque Nacional Nahuel Huapi para realizar actividades al aire libre como senderismo, avistamiento de aves o navegación por el lago\n- Visita al Cerro Campanario para disfrutar de una vista panorámica de la región\n\nDía 5:

## Sumando memoria

In [20]:
from langchain.prompts import (
    ChatPromptTemplate,
    MessagesPlaceholder,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

from langchain.memory import ConversationBufferMemory

prompt = ChatPromptTemplate(
    messages = [
    SystemMessagePromptTemplate.from_template("Tu eres un asistente que ayuda al usuario a planear y optimizar su itinerario"),
    MessagesPlaceholder(variable_name = 'chat_history'),
    HumanMessagePromptTemplate.from_template("{question}")
]
)


memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
conversation = LLMChain(
    llm=chat,
    prompt=prompt,
    verbose=False,
    memory=memory
)

/tmp/ipykernel_420324/3660914961.py:20: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  conversation = LLMChain(


In [22]:
while True:
    query = input('you: ')
    if query == 'q':
        break
    output = conversation({"question": query})
    print('User: ', query)
    print('AI system: ', output['text'])
    

User:  hola
AI system:  ¡Hola! ¿En qué puedo ayudarte hoy?
User:  
AI system:  ¿Cómo puedo ayudarte a planear tu día o tu viaje? ¿Necesitas información sobre destinos, recomendaciones de actividades o ayuda para organizar tu itinerario? ¡Estoy aquí para ayudarte!


### Sumando conocimiento no paramétricos

In [23]:
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.document_loaders import PyPDFLoader




In [27]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1500,
    chunk_overlap = 200
)

raw_documents = PyPDFLoader('../pdf/GuiaViajeBariloche.pdf').load()
documents = text_splitter.split_documents(raw_documents)
db = FAISS.from_documents(documents, OpenAIEmbeddings())

In [29]:
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(
            memory_key='chat_history',
            return_messages=True
        )

llm = ChatOpenAI()
qa_chain = ConversationalRetrievalChain.from_llm(llm, retriever=db.as_retriever(), memory=memory, verbose=True)
qa_chain.run({'question':'Dame información sobre circuito chico'})



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: Use the following pieces of context to answer the user's question. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
Sugerimos contratar  
 
 
Dónde Ir 
Circuito Chico 
Un recorrido de paisajes variados que se inicia por el borde del Lago Nahuel Huapi. Lo primero que 
encontrarás es Playa Bonita, de hermosas aguas azul profundo que se mezclan con el turquesa que 
llega a sus orillas. Si continuas el camino llegarás a Cerro Campanario, si te animas a llegar a su cumbre, 
podrás ver casi la totalidad de la región. Podes realiz ar este circuito a pie (hay senderos que guían tu 
camino) o a caballo. 
 
Circuito Grande 
Abarca 248 kilómetros de miradores, rocas y cerros, b osques tupidos. El punto de partida 
es Bariloche y ahí debes tomar la ruta Nº 237, hast a el puente del Río Limay. Hasta ahí te habrás 
maravillado 

'El Circuito Chico es un recorrido de paisajes variados en Bariloche que se inicia por el borde del Lago Nahuel Huapi. En este circuito, encontrarás lugares como Playa Bonita, con hermosas aguas azul profundo, y Cerro Campanario, desde donde se puede disfrutar de una vista panorámica de la región. Puedes realizar este circuito a pie o a caballo. ¡Es una experiencia imperdible para disfrutar de la naturaleza!'

In [31]:
from langchain.prompts.prompt import PromptTemplate

custom_template = """Dada la siguiente conversación y una pregunta complementaria, reformule la pregunta complementaria para que sea una pregunta independiente.
Si no encuentra la respuesta en el documento proporcionado, ignórelo y responda de todos modos.
Chat History:
{chat_history}
Follow Up Input: {question}
Standalone question:"""

CUSTOM_QUESTION_PROMPT = PromptTemplate.from_template(custom_template)

memory = ConversationBufferMemory(
            memory_key='chat_history',
            return_messages=True
        )


qa_chain = ConversationalRetrievalChain.from_llm(llm, retriever=db.as_retriever(), condense_question_prompt=CUSTOM_QUESTION_PROMPT, memory=memory, verbose=True)
qa_chain.run({'question':'que hay en estado unidos?'})



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: Use the following pieces of context to answer the user's question. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
Sugerimos contratar  
 
 
 
 
GUIA DE VIAJE 
BARILOCHE

Sugerimos contratar  
 
 
Bosque Tallado en El Bolsón 
A 120 kilómetros de Bariloche se encuentra El Bolsón , localidad que cuenta con el 
reconocido Bosque Tallado, lugar aprovechado por doce artistas, luego que fuese afectado por un 
incendio que dejó cientos de troncos dispersos por el l ugar, los que fueron tallados. Se trata de 
árboles en tonos amarillentos, bordeados por el Cerro  Piltriquitrón y el Río Quemquemtreu, que 
adornan, aún más, el paisaje.  
Cómo moverse 
En Bariloche la mayoría de las distancias son caminab les, en especial lo que es el centro. También 
existen dos empresas de colectivo que permiten movil izarse por el centro

'No tengo información sobre lugares en Estados Unidos, mi conocimiento se basa en la información proporcionada sobre Bariloche en Argentina. ¿Hay algo más en lo que pueda ayudarte?'

### Agente

In [33]:
from langchain.agents.agent_toolkits import create_retriever_tool

tool = create_retriever_tool(
    db.as_retriever(),
    "GuiaViajeBariloche",
    "Buscar y retornar documentos que sobre Bariloche"
)

tools = [tool]

memory = ConversationBufferMemory(
    memory_key='chat_history',
    return_messages=True
)

from langchain.agents.agent_toolkits import create_conversational_retrieval_agent

from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(openai_api_key=key, temperature = 0)

agent_executor = create_conversational_retrieval_agent(
    llm=llm,
    tools=tools,
    memory_key='chat_history',
    verbose=True
)


In [35]:
agent_executor({"input": "Hola, yo soy jose"})



> Entering new AgentExecutor chain...
Hola Jose! ¿En qué puedo ayudarte hoy?

> Finished chain.


{'input': 'Hola, yo soy jose',
 'chat_history': [HumanMessage(content="hi, i'm Vale", additional_kwargs={}, response_metadata={}),
  AIMessage(content='Hello Vale! How can I assist you today?', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='Hola, yo soy jose', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Hola Jose! ¿En qué puedo ayudarte hoy?', additional_kwargs={}, response_metadata={})],
 'output': 'Hola Jose! ¿En qué puedo ayudarte hoy?',
 'intermediate_steps': []}

In [36]:
agent_executor({"input": "Que me puedes decir de bariloche"})



> Entering new AgentExecutor chain...

Invoking: `GuiaViajeBariloche` with `{'query': 'Bariloche'}`


Bariloche cuenta con un importante número de museos,  mercados y ferias que dan cuenta de 
todo el valor histórico, artístico y artesanal de est as patagónicas tierras. Un clásico imperdible es 
el Museo de la Patagonia y el salón de exposiciones A sociación de Artesanos de Bariloche donde 
podrás apreciar y comprar los objetos creados por artistas de la zona. 
 
Expediciones 
Los sectores aledaños al centro de Bariloche son un t estimonio natural que debes conocer, tenes 
que recorrer los cerros, lagos y sus islas. Existen  dos circuitos; el Chico y Grande; los dos contemplan 
maravillosos paisajes naturales que podrás caminarlos a pie.

Sugerimos contratar  
 
 
Dónde Ir 
Circuito Chico 
Un recorrido de paisajes variados que se inicia por el borde del Lago Nahuel Huapi. Lo primero que 
encontrarás es Playa Bonita, de hermosas aguas azul profundo que se mezclan con el turquesa que 


{'input': 'Que me puedes decir de bariloche',
 'chat_history': [HumanMessage(content='Hola, yo soy jose', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Hola Jose! ¿En qué puedo ayudarte hoy?', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='Que me puedes decir de bariloche', additional_kwargs={}, response_metadata={}),
  AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"query":"Bariloche"}', 'name': 'GuiaViajeBariloche'}}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 145, 'total_tokens': 168, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-e3841b65-03b7-4a6b-b60f-42784e26e4fb-0'),
  FunctionMessage(content='Bariloche

In [37]:
agent_executor({"input": "que me puedes decir de jamica"})



> Entering new AgentExecutor chain...

Invoking: `GuiaViajeBariloche` with `{'query': 'Jamaica'}`


Sugerimos contratar  
 
 
 
 
GUIA DE VIAJE 
BARILOCHE

Sugerimos contratar  
 
 
BARILOCHE 
Bariloche ofrece una arquitectura única mezclando ma deras y piedras, facilidad para practicar 
deportes outdoors y aventuras, además probarás una gas tronomía excelente; mezcla europea y 
argentina, de donde nacen los chocolates más reconocidos de Argentina.  
Aire puro, paisajes recónditos y un pueblo en donde se  respira tranquilidad... Bariloche es mucho 
más que una pequeña ciudad en las tierras patagónicas . Es un mundo perfectamente ordenado y 
hermoso que debes conocer.  
 
Actividades Generales 
Deportes 
Gracias a su diversidad de paisajes y entornos natural es, Bariloche es una ciudad ideal para 
practicar todo tipo de deportes. Desde esquí en el invi erno, cuando las montañas de la Cordillera de 
los Andes posan blancas, hasta practicar trekking po r sus hermosos bosques. Sus lagos 

{'input': 'que me puedes decir de jamica',
 'chat_history': [HumanMessage(content='que me puedes decir de jamica', additional_kwargs={}, response_metadata={}),
  AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"query":"Jamaica"}', 'name': 'GuiaViajeBariloche'}}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 2074, 'total_tokens': 2097, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-dfca82b8-f75a-4277-9688-a240c56e01ed-0'),
  FunctionMessage(content='Sugerimos contratar  \n \n \n \n \nGUIA DE VIAJE \nBARILOCHE\n\nSugerimos contratar  \n \n \nBARILOCHE \nBariloche ofrece una arquitectura única mezclando ma deras y piedras, facilidad para practicar \ndeportes 

### Sumamos busqueda en internet

In [38]:
from langchain import SerpAPIWrapper
from langchain.agents import AgentType, initialize_agent
from langchain.llms import OpenAI
from langchain.tools import BaseTool, StructuredTool, Tool, tool

In [ ]:
key_serpapi = os.environ["SERPAPI_API_KEY"]
search = SerpAPIWrapper()

In [ ]:

tools = [
    Tool.from_function(
        func=search.run,
        name="Search",
        description="Útil para cuando necesitas responder preguntas sobre eventos actuales."
    ),
    create_retriever_tool(
    db.as_retriever(),
    "GuiaViajeBariloche",
    "Buscar y retornar documentos que sobre Bariloche"
    )
    ]

agent_executor = create_conversational_retrieval_agent(llm, tools, memory_key='chat_history', verbose=True)

In [ ]:
agent_executor({"input": "Cómo está el clima en bariloche?"})